# June 20

In [2]:
# Torch 
import torch
import torch.nn as nn

# MNIST1D 
import matplotlib.pyplot as plt 
import numpy as np 
from scipy.ndimage import gaussian_filter
from scipy.interpolate import interp1d 
from mnist1d.data import make_dataset, get_dataset_args

# Custom Modules 
import sys 
sys.path.append('../')
from Conv1d_NN import Conv1d_NN

from models import * 
from train import *

device = 'cpu'

In [4]:
# Conv1d_NN
model1 = nn.Sequential(
    Conv1d_NN(in_channels=1, out_channels =16, K = 3),  # Change the number of input channels to 1
    nn.ReLU(), 
    Conv1d_NN(in_channels=16, out_channels=16, K = 3),
    nn.ReLU(),
    Conv1d_NN(in_channels=16, out_channels=16, K = 3),
    nn.ReLU(),
    nn.Flatten(), 
    nn.Linear(640, 10)
)

# Get summary 
from torchsummary import summary
summary(model1, (1, 40), device=device)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 16, 40]              64
         Conv1d_NN-2               [-1, 16, 40]               0
              ReLU-3               [-1, 16, 40]               0
            Conv1d-4               [-1, 16, 40]             784
         Conv1d_NN-5               [-1, 16, 40]               0
              ReLU-6               [-1, 16, 40]               0
            Conv1d-7               [-1, 16, 40]             784
         Conv1d_NN-8               [-1, 16, 40]               0
              ReLU-9               [-1, 16, 40]               0
          Flatten-10                  [-1, 640]               0
           Linear-11                   [-1, 10]           6,410
Total params: 8,042
Trainable params: 8,042
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/back

In [6]:
# Conv1d_NN
model2 = nn.Sequential(
    Conv1d_NN(in_channels=1, out_channels =8, K = 3),  # Change the number of input channels to 1
    nn.ReLU(), 
    Conv1d_NN(in_channels=8, out_channels=8, K = 3),
    nn.ReLU(),
    Conv1d_NN(in_channels=8, out_channels=8, K = 3),
    nn.ReLU(),
    nn.Flatten(), 
    nn.Linear(320, 10)
)

# Get summary 
from torchsummary import summary
summary(model2, (1, 40), device=device)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1                [-1, 8, 40]              32
         Conv1d_NN-2                [-1, 8, 40]               0
              ReLU-3                [-1, 8, 40]               0
            Conv1d-4                [-1, 8, 40]             200
         Conv1d_NN-5                [-1, 8, 40]               0
              ReLU-6                [-1, 8, 40]               0
            Conv1d-7                [-1, 8, 40]             200
         Conv1d_NN-8                [-1, 8, 40]               0
              ReLU-9                [-1, 8, 40]               0
          Flatten-10                  [-1, 320]               0
           Linear-11                   [-1, 10]           3,210
Total params: 3,642
Trainable params: 3,642
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/back

In [9]:
# Conv1d_NN
model3 = nn.Sequential(
    Conv1d_NN(in_channels=1, out_channels =4, K = 3),  # Change the number of input channels to 1
    nn.ReLU(), 
    Conv1d_NN(in_channels=4, out_channels=4, K = 3),
    nn.ReLU(),
    Conv1d_NN(in_channels=4, out_channels=4, K = 3),
    nn.ReLU(),
    nn.Flatten(), 
    nn.Linear(160, 10)
)

# Get summary 
from torchsummary import summary
summary(model3, (1, 40), device=device)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1                [-1, 4, 40]              16
         Conv1d_NN-2                [-1, 4, 40]               0
              ReLU-3                [-1, 4, 40]               0
            Conv1d-4                [-1, 4, 40]              52
         Conv1d_NN-5                [-1, 4, 40]               0
              ReLU-6                [-1, 4, 40]               0
            Conv1d-7                [-1, 4, 40]              52
         Conv1d_NN-8                [-1, 4, 40]               0
              ReLU-9                [-1, 4, 40]               0
          Flatten-10                  [-1, 160]               0
           Linear-11                   [-1, 10]           1,610
Total params: 1,730
Trainable params: 1,730
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/back

In [10]:
# Data


from mnist1d.data import make_dataset, get_dataset_args

less_noise_args = get_dataset_args(as_dict=False)
# {'num_samples': 5000, 'train_split': 0.8, 'template_len': 12, 
# 'padding': [36, 60], 'scale_coeff': 0.4, 'max_translation': 48, 
# 'corr_noise_scale': 0.25, 'iid_noise_scale': 0.02, 'shear_scale': 0.75, 
# 'shuffle_seq': False, 'final_seq_length': 40, 'seed': 42, 
# 'url': 'https://github.com/greydanus/mnist1d/raw/master/mnist1d_data.pkl'}

set_seed(less_noise_args.seed)
less_noise_args.shuffle_seq = False
less_noise_args.corr_noise_scale = 0.00
less_noise_args.iid_noise_scale = 0.000



less_noise_data = make_dataset(args = less_noise_args) # Make the dataset 
less_noise_data['x'] = torch.Tensor(less_noise_data['x']).unsqueeze(1)
less_noise_data['x_test'] = torch.Tensor(less_noise_data['x_test']).unsqueeze(1)

print(less_noise_args.corr_noise_scale, less_noise_args.iid_noise_scale)
print(less_noise_data['x'].shape, less_noise_data['y'].shape, less_noise_data['t'].shape)

0.0 0.0
torch.Size([4000, 1, 40]) (4000,) (40,)


In [11]:
less_noise_model = get_model_args(as_dict=False)
less_noise_model.device = 'mps'
less_noise_model.total_steps = 6000

print("16 Channels")
results =train_model(less_noise_data, model1, less_noise_model)
print()

print("8 Channels")
results =train_model(less_noise_data, model2, less_noise_model)
print()

print("4 Channels")
results =train_model(less_noise_data, model3, less_noise_model)
print()

16 Channels
step 1000, dt 19.47s, train_loss 4.725e-01, test_loss 6.223e-01, train_acc 85.3, test_acc 74.7
step 2000, dt 16.64s, train_loss 6.130e-01, test_loss 7.010e-01, train_acc 86.8, test_acc 75.4
step 3000, dt 16.92s, train_loss 2.406e-01, test_loss 6.678e-01, train_acc 90.6, test_acc 77.9
step 4000, dt 16.68s, train_loss 3.853e-01, test_loss 8.004e-01, train_acc 87.0, test_acc 73.4
step 5000, dt 16.56s, train_loss 2.743e-01, test_loss 8.001e-01, train_acc 89.5, test_acc 75.1
step 6000, dt 16.60s, train_loss 3.055e-01, test_loss 7.177e-01, train_acc 88.9, test_acc 75.5

8 Channels
step 1000, dt 16.68s, train_loss 5.475e-01, test_loss 7.123e-01, train_acc 76.4, test_acc 69.8
step 2000, dt 15.95s, train_loss 4.535e-01, test_loss 6.019e-01, train_acc 82.7, test_acc 76.5
step 3000, dt 16.25s, train_loss 4.108e-01, test_loss 6.158e-01, train_acc 82.9, test_acc 76.4
step 4000, dt 15.82s, train_loss 3.938e-01, test_loss 5.318e-01, train_acc 87.2, test_acc 78.4
step 5000, dt 17.28s, trai